In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [42]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

In [43]:
categorical_cols = ['cat'+str(i) for i in range(19)]
continous_cols = ['cont'+str(i) for i in range(11)]


In [44]:
cols=categorical_cols+continous_cols
train_objs_num = len(train)
dataset = pd.concat(objs=[train[cols], test[cols]], axis=0)
dataset_preprocessed = pd.get_dummies(dataset,columns=categorical_cols)
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

In [45]:
train_preprocessed.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,cat16_C,cat16_D,cat17_A,cat17_B,cat17_C,cat17_D,cat18_A,cat18_B,cat18_C,cat18_D
0,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,...,0,1,0,0,0,1,0,1,0,0
1,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,...,0,0,0,0,0,1,0,1,0,0
2,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,...,0,1,0,0,0,1,0,1,0,0
3,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,...,0,1,0,0,0,1,0,1,0,0
4,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,...,0,0,0,0,0,1,0,1,0,0


<h3>Denoising Auto Encoder</h3>

In [46]:
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras import Model
from  tensorflow.keras.regularizers import l2
import tensorflow as tf

In [47]:
def DAutoEnc():
    # denoising autoencoder
    inputs = Input((11,))
    x = Dense(1500, activation='relu')(inputs) # 1500 original
    x = Dense(1500, activation='relu', name="feature")(x) # 1500 original
    x = Dense(1500, activation='relu')(x) # 1500 original
    outputs = Dense(11, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    return model

In [48]:
alldata = pd.concat([train[continous_cols],test[continous_cols]],axis=0)
print(alldata.shape)
autoencoder = DAutoEnc()
autoencoder.fit(alldata[continous_cols], alldata[continous_cols],
                    epochs=10,
                    batch_size=256,
                    shuffle=True
                    )

(500000, 11)
Epoch 1/10
1954/1954 [==============================] - 7s 4ms/step - loss: 0.0317
Epoch 2/10
1954/1954 [==============================] - 6s 3ms/step - loss: 0.0220
Epoch 3/10
1954/1954 [==============================] - 7s 3ms/step - loss: 2.1966e-05
Epoch 4/10
1954/1954 [==============================] - 6s 3ms/step - loss: 3.0137e-05
Epoch 5/10
1954/1954 [==============================] - 7s 3ms/step - loss: 2.8383e-05
Epoch 6/10
1954/1954 [==============================] - 7s 4ms/step - loss: 2.7551e-05
Epoch 7/10
1954/1954 [==============================] - 6s 3ms/step - loss: 2.4597e-05
Epoch 8/10
1954/1954 [==============================] - 7s 3ms/step - loss: 2.3474e-05
Epoch 9/10
1954/1954 [==============================] - 6s 3ms/step - loss: 2.1552e-05
Epoch 10/10
1954/1954 [==============================] - 7s 3ms/step - loss: 1.9738e-05


In [49]:
test_denoised = test_preprocessed.copy()
test_denoised[continous_cols] = autoencoder.predict(test_denoised[continous_cols])
train_denoised = train_preprocessed.copy()
train_denoised[continous_cols] = autoencoder.predict(train_denoised[continous_cols])

In [50]:
train_denoised['target'] = train.target

In [51]:
X_train = train_denoised.drop(['target'], axis=1)
y_train = train_denoised.target
X_test = test_denoised


X_train.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,cat16_C,cat16_D,cat17_A,cat17_B,cat17_C,cat17_D,cat18_A,cat18_B,cat18_C,cat18_D
0,0.616541,0.856807,0.748349,0.795595,0.656140,0.624818,0.585004,0.792172,0.804156,0.947893,...,0,1,0,0,0,1,0,1,0,0
1,0.363564,0.333548,0.380106,0.541725,0.374268,0.361443,0.597770,0.410456,0.394388,0.916911,...,0,0,0,0,0,1,0,1,0,0
2,0.501419,0.323008,0.335351,0.614800,0.779011,0.556861,0.344301,0.394351,0.404020,0.278459,...,0,1,0,0,0,1,0,1,0,0
3,0.926814,0.707978,0.818345,0.803274,0.784809,0.624543,0.215129,0.899929,0.618836,0.741932,...,0,1,0,0,0,1,0,1,0,0
4,0.250592,0.273631,0.332015,0.279375,0.598534,0.130255,0.576513,0.281214,0.346019,0.347752,...,0,0,0,0,0,1,0,1,0,0


In [52]:
print(X_train.shape,y_train.shape)

(300000, 642) (300000,)


In [53]:
X_train,X_val=X_train.iloc[:-2000,:],X_train.iloc[-2000:,:]


In [54]:
y_train,y_val=y_train.iloc[:-2000],y_train.iloc[-2000:]

In [55]:
print(X_train.shape,y_train.shape)

(298000, 642) (298000,)


In [56]:
import random
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import optuna
BEST_PARAMS = {'learning_rate': 0.03, 'metric': 'auc'}
K=5
# Objective function
def objective(trial):
    # Search spaces
    hyperparams = {
        'n_estimators': trial.suggest_int('n_estimators', 100,500),
        'num_leaves': trial.suggest_int('num_leaves', 2, 131072),
        'max_depth': trial.suggest_int('max_depth', 1, 32),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1E-16, 25),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1E-16, 25),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 1E-16, 1.0),
#         'subsample': trial.suggest_float('subsample ', 1E-16, 1.0),
#         'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 50.0)  
    }
    lgb=LGBMClassifier(**hyperparams)
    lgb.fit(X_train,y_train,eval_set=(X_val,y_val),eval_metric='auc',early_stopping_rounds=100,verbose=False)
    predictions=lgb.predict_proba(X_val)[:,1]

    return roc_auc_score(y_val,predictions)




In [57]:
# Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=1200)

[I 2021-03-09 05:49:08,672] A new study created in memory with name: no-name-c812d91f-d4ca-4514-9b5a-f5e1afdf0c24
[I 2021-03-09 05:49:28,444] Trial 0 finished with value: 0.9002983174192963 and parameters: {'n_estimators': 341, 'num_leaves': 111493, 'max_depth': 8, 'colsample_bytree': 0.2293293038739039}. Best is trial 0 with value: 0.9002983174192963.
[I 2021-03-09 05:50:20,587] Trial 1 finished with value: 0.8887091071740872 and parameters: {'n_estimators': 305, 'num_leaves': 16398, 'max_depth': 18, 'colsample_bytree': 0.8930529313666554}. Best is trial 0 with value: 0.9002983174192963.
[I 2021-03-09 05:51:55,694] Trial 2 finished with value: 0.8910455704707275 and parameters: {'n_estimators': 449, 'num_leaves': 42610, 'max_depth': 29, 'colsample_bytree': 0.04391408676318962}. Best is trial 0 with value: 0.9002983174192963.
[I 2021-03-09 05:59:25,695] Trial 3 finished with value: 0.8881605352853769 and parameters: {'n_estimators': 176, 'num_leaves': 86657, 'max_depth': 32, 'colsample

In [58]:
params=study.best_params
study.best_params

In [63]:
params

{'n_estimators': 341,
 'num_leaves': 111493,
 'max_depth': 8,
 'colsample_bytree': 0.2293293038739039}

In [59]:
model=LGBMClassifier(**params) 
model.fit(X_train,y_train)
preds=model.predict(X_test)
preds.shape

(200000,)

In [60]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
sub.head()
sub.shape

(200000, 2)

In [62]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)